In [1]:
################################################# mitiq functions
from mitiq import zne
from mitiq.zne import mitigate_executor
from mitiq.zne.inference import RichardsonFactory
################################################# braket functions
import braket
from braket.devices import LocalSimulator
from braket.circuits import Circuit, Noise, Observable
################################################# other packages
import numpy as np

################################################# ideal density matrix |ψ⟩⟨ψ|
def density_matrix_pure(state_vector: np.ndarray) -> np.ndarray:
    return np.outer(state_vector, np.transpose(np.conjugate(state_vector)))
bell_state = np.array([1/np.sqrt(2),0,0,1/np.sqrt(2)])
ideal_target_bell = density_matrix_pure(bell_state)

################################################# circuit for Bell state
circuit = Circuit()
circuit.h(0).cnot(0,1)
print(circuit)

T  : |0|1|
          
q0 : -H-C-
        | 
q1 : ---X-

T  : |0|1|


In [3]:
def noiseless_executor(circ: Circuit) -> float:
    device = LocalSimulator('braket_dm') 
    circ.expectation(observable=Observable.Hermitian(matrix=ideal_target_bell), target=range(2))
    task = device.run(circ)
    result = task.result()
    return result.values

def executor_with_noise(circ: Circuit) -> float:
    noise = Noise.Depolarizing(probability=0.04)
    circ.apply_gate_noise(noise)
    return noiseless_executor(circ)

# noise_fidelity = np.sqrt(executor_with_noise(circuit))
# print(noise_fidelity)

In [4]:
fac = RichardsonFactory(scale_factors=[1, 3, 5])
mitigated_executor = mitigate_executor(executor_with_noise, factory=fac)
mitigated_fidelity = np.sqrt(mitigated_executor(circuit))
print(mitigated_fidelity)

C:\Users\nadim\anaconda3\envs\env_mitiq\Lib\site-packages\mitiq\zne\inference.py:97: UserWarning: The input circuit is very short. This may reduce the accuracy of noise scaling.
  warnings.warn(


0.9915175318851517


In [7]:
################################################# qiskit functions
from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp
from qiskit_ibm_runtime import QiskitRuntimeService, Session, Options, Estimator

################################################# ideal density matrix |ψ⟩⟨ψ|
observable = SparsePauliOp("XX", coeffs = 1/4)+SparsePauliOp("YY", coeffs = -1/4)+ SparsePauliOp("ZZ", coeffs = 1/4)+SparsePauliOp("II", coeffs = 1/4)

################################################# circuit for Bell state
circuit = QuantumCircuit(2) 
circuit.h(0)
circuit.cx(0,1)

In [ ]:
backend = "ibmq_manila" # you need your IBM quantum account to use this! Have a look here https://quantum-computing.ibm.com/
service = QiskitRuntimeService()

with Session(service=service, backend=backend) as session:
    options = Options()
    options.resilience_level = 2
    est = Estimator(session=session, options=options)
    job = est.run(circuit, observable)

result = job.result()
print(f"  > Expectation value: {result.values[0]}")

In [ ]:
backend = "ibm_lagos" # you need your IBM quantum account to use this! Have a look here https://quantum-computing.ibm.com/
service = QiskitRuntimeService()

with Session(service=service, backend=backend) as session:
    options = Options()
    options.resilience_level = 2
    est = Estimator(session=session, options=options)
    job = est.run(circuit, observable)

result = job.result()
print(f"  > Expectation value: {result.values[0]}")